In [114]:
import pandas as pd
import numpy as np
import joblib
import warnings
warnings.filterwarnings('ignore')

In [115]:
user_info = pd.read_csv('./user_info.csv')
user_info['reg_date'] = pd.to_datetime(user_info['reg_date']).dt.date

In [116]:
user_info.head()

,user_id,reg_date,mobile,user_type,user_flag,platform,channel_id,source_id,user_creates_3_server_num,user_creates_3_role_num
0,45035796,2020-04-14,NaN,2,1,1,639,576,NaN,NaN
1,45051454,2020-03-16,NaN,1,1,1,639,576,1.0,1.0
2,45059816,2019-05-24,NaN,2,1,2,20021,22791,NaN,NaN
3,45130799,2020-07-06,NaN,1,1,1,639,576,1.0,1.0
4,45208326,2020-05-23,NaN,1,1,1,639,576,NaN,NaN


In [117]:
source_1 = pd.read_csv('./source_1.csv')
source_2 = pd.read_csv('./source_2.csv')

In [118]:
source_1.rename(columns={'chl_user_id':'channel_id'},inplace=True)
source_1.drop_duplicates(subset=['channel_id','source_id'],inplace=True)
source = pd.merge(source_1,source_2,on=['channel_id','source_id'],how='left',validate='one_to_many')
source.dropna(inplace=True)

In [119]:
source['create_time'] = pd.to_datetime(source['create_time']).dt.date
source['tdate'] = pd.to_datetime(source['tdate']).dt.date

source.rename(columns={'tdate':'reg_date'},inplace=True)

In [120]:
# 统计cpc(计划创角成本)
source['cpc'] = source['amount']/(source['create_role_num']+1e-4)

In [56]:
source.head()

,channel_id,source_id,create_time,reg_date,amount,paper_use_amount,click_num,down_num,pv,create_role_num,cpc
0,20953,252731,2020-01-02,2020-01-02,1562.56,1562.56,1552.0,55.0,53605.0,51.0,30.638371
1,20953,252731,2020-01-02,2020-01-03,0.71,0.71,6.0,3.0,1054.0,6.0,0.118331
2,20953,252731,2020-01-02,2020-01-05,0.00,0.00,0.0,2.0,0.0,1.0,0.000000
3,20953,252731,2020-01-02,2020-01-08,0.00,0.00,0.0,0.0,0.0,1.0,0.000000
4,20953,252731,2020-01-02,2020-01-11,0.00,0.00,0.0,1.0,0.0,1.0,0.000000


In [121]:
source[(source['create_role_num']==0)&(source['amount']!=0)]

,channel_id,source_id,create_time,reg_date,amount,paper_use_amount,click_num,down_num,pv,create_role_num,cpc
397,20754,253111,2020-01-02,2020-01-04,0.88,0.88,7.0,0.0,321.0,0.0,8800.0
1073,20953,253653,2020-01-04,2020-01-05,54.94,54.94,53.0,0.0,5634.0,0.0,549400.0
1075,20953,253653,2020-01-04,2020-01-07,3.58,3.58,4.0,0.0,130.0,0.0,35800.0
1138,20754,253704,2020-01-04,2020-01-04,556.06,556.06,185.0,0.0,17035.0,0.0,5560600.0
1197,20754,253714,2020-01-04,2020-01-04,190.68,190.68,14.0,0.0,4973.0,0.0,1906800.0
...,...,...,...,...,...,...,...,...,...,...,...
278903,20646,334706,2020-07-22,2020-07-23,0.13,0.17,1.0,0.0,293.0,0.0,1300.0
278931,20646,334881,2020-07-23,2020-07-23,26.64,35.16,184.0,0.0,11692.0,0.0,266400.0
279023,20646,336221,2020-07-27,2020-07-28,0.23,0.31,1.0,0.0,823.0,0.0,2300.0
279034,20646,336224,2020-07-27,2020-07-28,41.61,54.93,158.0,0.0,65517.0,0.0,416100.0


In [122]:
source.drop_duplicates(subset=['channel_id','source_id','reg_date'],inplace=True)

In [123]:
source.shape

(275486, 11)

In [59]:
source.to_csv('./source.csv')

In [124]:
source_notna = source[source['amount'] != 0]

In [125]:
source_notna['amount'].describe()

count     69686.000000
mean       2165.596918
std        5564.881730
min           0.010000
25%         134.032500
50%         598.320000
75%        2132.702500
max      209752.390000
Name: amount, dtype: float64

In [126]:
source_notna.shape

(69686, 11)

In [127]:
user_info = user_info[['user_id','channel_id','source_id','reg_date']]
user_info.drop_duplicates(inplace=True)

In [128]:
user_info.shape

(30226034, 4)

In [129]:
user_info.head()

,user_id,channel_id,source_id,reg_date
0,45035796,639,576,2020-04-14
1,45051454,639,576,2020-03-16
2,45059816,20021,22791,2019-05-24
3,45130799,639,576,2020-07-06
4,45208326,639,576,2020-05-23


In [130]:
source_info = pd.merge(source,user_info,on=['channel_id','source_id','reg_date'],how='left',validate='one_to_many')

In [131]:
source_info.dropna(inplace=True)

In [132]:
source_info.shape

(5679074, 12)

In [133]:
source_info.head()

,channel_id,source_id,create_time,reg_date,amount,paper_use_amount,click_num,down_num,pv,create_role_num,cpc,user_id
0,20953,252731,2020-01-02,2020-01-02,1562.56,1562.56,1552.0,55.0,53605.0,51.0,30.638371,100261967.0
1,20953,252731,2020-01-02,2020-01-02,1562.56,1562.56,1552.0,55.0,53605.0,51.0,30.638371,100262535.0
2,20953,252731,2020-01-02,2020-01-02,1562.56,1562.56,1552.0,55.0,53605.0,51.0,30.638371,100268013.0
3,20953,252731,2020-01-02,2020-01-02,1562.56,1562.56,1552.0,55.0,53605.0,51.0,30.638371,100269429.0
4,20953,252731,2020-01-02,2020-01-02,1562.56,1562.56,1552.0,55.0,53605.0,51.0,30.638371,100273946.0


In [ ]:
# source_info.set_index(['channel_id','source_id'],drop=False,inplace=True)

In [134]:
role_info_1d = pd.read_pickle('./role_info_1d.pkl')

In [135]:
role_info_1d.shape

(5141450, 24)

In [136]:
source_info['user_id'] = source_info['user_id'].astype(np.int64)
role_info_1d['user_id'] = role_info_1d['user_id'].astype(np.int64)
data_1 = pd.merge(source_info,role_info_1d,on='user_id',how='left',validate='one_to_many')
data_1.dropna(inplace=True)

In [137]:
# 统计每条计划的30天回款source_pay_sum
group_1 = data_1.groupby(['channel_id','source_id','reg_date'])
temp = group_1['role_created_30_pay_sum'].agg([('source_pay_sum', 'sum')]).reset_index()
data_1 = pd.merge(data_1,temp,on=['channel_id','source_id','reg_date'],how='left',validate='many_to_one')

In [138]:
pd.set_option('display.max_columns',None)
data_1.head()

,channel_id,source_id,create_time,reg_date,amount,paper_use_amount,click_num,down_num,pv,create_role_num,cpc,user_id,cp_server_no,cp_role_id,role_created_login_num,role_created_active,role_created_online,max_role_level,ip_num,pay_num,pay_sum,active_0-8,active_8-12,active_12-14,active_14-18,active_18-24,pay_grade_1,pay_grade_2,pay_grade_3,pay_grade_4,pay_grade_5,pay_grade_6,pay_rate,pay_avg,role_created_30_pay_sum,source_pay_sum
0,20953,252731,2020-01-02,2020-01-02,1562.56,1562.56,1552.0,55.0,53605.0,51.0,30.638371,100261967,660192,13814061,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,20953,252731,2020-01-02,2020-01-02,1562.56,1562.56,1552.0,55.0,53605.0,51.0,30.638371,100262535,660191,13814172,16.0,1.0,34068.0,44.0,1.0,0.0,0.0,40.0,22.0,6.0,13.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20953,252731,2020-01-02,2020-01-02,1562.56,1562.56,1552.0,55.0,53605.0,51.0,30.638371,100262535,660192,13814172,9.0,1.0,29334.0,42.0,1.0,0.0,0.0,14.0,0.0,0.0,20.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20953,252731,2020-01-02,2020-01-02,1562.56,1562.56,1552.0,55.0,53605.0,51.0,30.638371,100262535,660193,13814172,4.0,1.0,6350.0,36.0,1.0,0.0,0.0,8.0,6.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20953,252731,2020-01-02,2020-01-02,1562.56,1562.56,1552.0,55.0,53605.0,51.0,30.638371,100268013,660192,13815011,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [139]:
data_1.shape

(3992564, 36)

In [ ]:
data_1[role_info_3d['role_created_30_pay_sum']!=0]

## 预测

In [140]:
df_pay = role_info_1d[role_info_1d['pay_sum']!=0]
df_nopay = role_info_1d[role_info_1d['pay_sum']==0]

In [141]:
df_nopay_online_n = df_nopay[(df_nopay['role_created_online']<=600) & (df_nopay['role_created_login_num']<=2)&(df_nopay['max_role_level']<=10)]
df_nopay_online_y = df_nopay.loc[np.setdiff1d(df_nopay.index, df_nopay_online_n.index),:]

In [142]:
# 标记角户质量不付费不在线0;不付费在线1;付费2
df_nopay_online_n['role_type'] = 0
df_nopay_online_y['role_type'] = 1
df_pay['role_type'] = 2

In [143]:
print('0类的角色占比：%.4f' % (df_nopay_online_n.shape[0]/role_info_1d.shape[0]))
print('1类的角色占比：%.4f' % (df_nopay_online_y.shape[0]/role_info_1d.shape[0]))
print('2类的角色占比：%.4f' % (df_pay.shape[0]/role_info_1d.shape[0]))

0类的角色占比：0.6122
1类的角色占比：0.3723
2类的角色占比：0.0155


In [144]:
df_nopay_online_n['predict_30_pay'] = 0
df_nopay_online_y['predict_30_pay'] = 0

In [145]:
lgb_r_1d = joblib.load('./lgb_r_1d.pkl')

In [146]:
df_pay['predict_30_pay'] = np.expm1(lgb_r_1d.predict(df_pay.drop(['user_id','cp_server_no','role_type',
                                                            'cp_role_id','role_created_30_pay_sum'],axis=1)))*2.1

In [147]:
df_pay['predict_30_pay'].sum()/role_info_1d['role_created_30_pay_sum'].sum()

0.8452350400226288

In [149]:
df_predict = pd.concat([df_pay,df_nopay_online_n,df_nopay_online_y],axis=0)

In [150]:
df_predict.head()

,user_id,cp_server_no,cp_role_id,role_created_login_num,role_created_active,role_created_online,max_role_level,ip_num,pay_num,pay_sum,active_0-8,active_8-12,active_12-14,active_14-18,active_18-24,pay_grade_1,pay_grade_2,pay_grade_3,pay_grade_4,pay_grade_5,pay_grade_6,pay_rate,pay_avg,role_created_30_pay_sum,role_type,predict_30_pay
19,102551485,14403653,14403653,51.0,1.0,34815.0,1.0,1.0,1.0,12.0,27,37,29,30,32,0,1,0,0,0,0,0.9999,11.998800,218.0,2,100.952175
163,102942389,14503750,14503750,32.0,1.0,14152.0,39.0,1.0,1.0,12.0,34,0,15,0,19,0,1,0,0,0,0,0.9999,11.998800,210.0,2,63.406752
191,102968330,14509735,14509735,38.0,1.0,27311.0,1.0,2.0,3.0,110.0,24,17,2,21,59,0,2,1,0,0,0,2.9997,36.665444,256.0,2,971.444733
214,102987110,14514764,14514764,52.0,1.0,72236.0,1.0,3.0,4.0,200.0,8,27,10,63,16,0,3,0,1,0,0,3.9996,49.998750,444.0,2,1777.523515
262,103029390,14524806,14524806,38.0,1.0,15612.0,44.0,2.0,2.0,42.0,0,19,5,7,59,0,2,0,0,0,0,1.9998,20.998950,42.0,2,266.128208


In [151]:
source_info['user_id'] = source_info['user_id'].astype(np.int64)
data = pd.merge(source_info,df_predict,on='user_id',how='left',validate='one_to_many')
data.dropna(inplace=True)

In [152]:
data.shape

(3992564, 37)

In [153]:
pd.set_option('display.max_columns',None)
data.head()

,channel_id,source_id,create_time,reg_date,amount,paper_use_amount,click_num,down_num,pv,create_role_num,cpc,user_id,cp_server_no,cp_role_id,role_created_login_num,role_created_active,role_created_online,max_role_level,ip_num,pay_num,pay_sum,active_0-8,active_8-12,active_12-14,active_14-18,active_18-24,pay_grade_1,pay_grade_2,pay_grade_3,pay_grade_4,pay_grade_5,pay_grade_6,pay_rate,pay_avg,role_created_30_pay_sum,role_type,predict_30_pay
0,20953,252731,2020-01-02,2020-01-02,1562.56,1562.56,1552.0,55.0,53605.0,51.0,30.638371,100261967,660192,13814061,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,20953,252731,2020-01-02,2020-01-02,1562.56,1562.56,1552.0,55.0,53605.0,51.0,30.638371,100262535,660191,13814172,16.0,1.0,34068.0,44.0,1.0,0.0,0.0,40.0,22.0,6.0,13.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20953,252731,2020-01-02,2020-01-02,1562.56,1562.56,1552.0,55.0,53605.0,51.0,30.638371,100262535,660192,13814172,9.0,1.0,29334.0,42.0,1.0,0.0,0.0,14.0,0.0,0.0,20.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,20953,252731,2020-01-02,2020-01-02,1562.56,1562.56,1552.0,55.0,53605.0,51.0,30.638371,100262535,660193,13814172,4.0,1.0,6350.0,36.0,1.0,0.0,0.0,8.0,6.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,20953,252731,2020-01-02,2020-01-02,1562.56,1562.56,1552.0,55.0,53605.0,51.0,30.638371,100268013,660192,13815011,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [154]:
groups = data.groupby(['channel_id','source_id','reg_date'])

In [155]:
groups.ngroups

79849

In [156]:
# 统计计划（天）的30天真实回款
temp = groups['role_created_30_pay_sum'].agg([('source_30_pay_sum','sum')]).reset_index()
df = pd.merge(data,temp,on=['channel_id','source_id','reg_date'],how='left',validate='many_to_one')

In [157]:
# 统计计划（天）的30天预测回款
temp = groups['predict_30_pay'].agg([('source_30_predict_pay','sum')]).reset_index()
df = pd.merge(df,temp,on=['channel_id','source_id','reg_date'],how='left',validate='many_to_one')

In [187]:
def count_0(list):
    count = 0
    for num in list:
        if num ==0:
            count += 1
    return count

def count_1(list):
    count = 0
    for num in list:
        if num ==1:
            count += 1
    return count

def count_2(list):
    count = 0
    for num in list:
        if num ==2:
            count += 1
    return count

In [212]:
temp = groups['role_type'].apply(count_0).reset_index().rename(columns = {'role_type':'type_0_num'})
temp['reg_date'] = temp['reg_date'].astype(str)
df['reg_date'] = df['reg_date'].astype(str)
df = pd.merge(df,temp,on=['channel_id','source_id','reg_date'],how='left',validate='many_to_one')

In [216]:
temp = groups['role_type'].apply(count_1).reset_index().rename(columns = {'role_type':'type_1_num'})
temp['reg_date'] = temp['reg_date'].astype(str)
df['reg_date'] = df['reg_date'].astype(str)
df = pd.merge(df,temp,on=['channel_id','source_id','reg_date'],how='left',validate='many_to_one')

In [217]:
temp = groups['role_type'].apply(count_2).reset_index().rename(columns = {'role_type':'type_2_num'})
temp['reg_date'] = temp['reg_date'].astype(str)
df['reg_date'] = df['reg_date'].astype(str)
df = pd.merge(df,temp,on=['channel_id','source_id','reg_date'],how='left',validate='many_to_one')

In [219]:
df['type_0_rate'] = df['type_0_num']/(df['type_0_num']+df['type_1_num']+df['type_2_num'])
df['type_1_rate'] = df['type_1_num']/(df['type_0_num']+df['type_1_num']+df['type_2_num'])
df['type_2_rate'] = df['type_2_num']/(df['type_0_num']+df['type_1_num']+df['type_2_num'])
df['source_role_num'] = df['type_0_num']+df['type_1_num']+df['type_2_num']

In [220]:
df.head()

,channel_id,source_id,create_time,reg_date,amount,paper_use_amount,click_num,down_num,pv,create_role_num,user_id,cp_server_no,cp_role_id,role_created_login_num,role_created_active,role_created_online,max_role_level,ip_num,pay_num,pay_sum,active_0-8,active_8-12,active_12-14,active_14-18,active_18-24,pay_grade_1,pay_grade_2,pay_grade_3,pay_grade_4,pay_grade_5,pay_grade_6,pay_rate,pay_avg,role_created_30_pay_sum,role_type,predict_30_pay,source_30_pay_sum,source_30_predict_pay,type_0_num,type_1_num,type_2_num,type_0_rate,type_1_rate,type_2_rate,source_role_num
0,20953,252731,2020-01-02,2020-01-02,1562.56,1562.56,1552.0,55.0,53605.0,51.0,100261967,660192,13814061,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,48,3,0,0.941176,0.058824,0.0,51
1,20953,252731,2020-01-02,2020-01-02,1562.56,1562.56,1552.0,55.0,53605.0,51.0,100262535,660191,13814172,16.0,1.0,34068.0,44.0,1.0,0.0,0.0,40.0,22.0,6.0,13.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,48,3,0,0.941176,0.058824,0.0,51
2,20953,252731,2020-01-02,2020-01-02,1562.56,1562.56,1552.0,55.0,53605.0,51.0,100262535,660192,13814172,9.0,1.0,29334.0,42.0,1.0,0.0,0.0,14.0,0.0,0.0,20.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,48,3,0,0.941176,0.058824,0.0,51
3,20953,252731,2020-01-02,2020-01-02,1562.56,1562.56,1552.0,55.0,53605.0,51.0,100262535,660193,13814172,4.0,1.0,6350.0,36.0,1.0,0.0,0.0,8.0,6.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,48,3,0,0.941176,0.058824,0.0,51
4,20953,252731,2020-01-02,2020-01-02,1562.56,1562.56,1552.0,55.0,53605.0,51.0,100268013,660192,13815011,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,48,3,0,0.941176,0.058824,0.0,51


In [225]:
df_source = df.drop_duplicates(subset=['channel_id','source_id','reg_date'])
col = ['channel_id','source_id','create_time','reg_date','amount','paper_use_amount','click_num','down_num','pv',
       'create_role_num','source_30_pay_sum','source_30_predict_pay','type_0_num','type_1_num','type_2_num','type_0_rate',
       'type_1_rate','type_2_rate','source_role_num']
df_source = df_source[col]

In [226]:
df_source.shape

(79849, 19)

In [228]:
df_source['30_ROI'] = df_source['source_30_pay_sum']/df_source['amount']

In [242]:
df_source['30_ROI_predict'] = df_source['source_30_predict_pay']/df_source['amount']

In [243]:
df_source.head()

,channel_id,source_id,create_time,reg_date,amount,paper_use_amount,click_num,down_num,pv,create_role_num,source_30_pay_sum,source_30_predict_pay,type_0_num,type_1_num,type_2_num,type_0_rate,type_1_rate,type_2_rate,source_role_num,30_ROI,30_ROI_predict
0,20953,252731,2020-01-02,2020-01-02,1562.56,1562.56,1552.0,55.0,53605.0,51.0,0.0,0.0,48,3,0,0.941176,0.058824,0.0,51,0.0,0.0
51,20953,252731,2020-01-02,2020-01-03,0.71,0.71,6.0,3.0,1054.0,6.0,0.0,0.0,6,0,0,1.000000,0.000000,0.0,6,0.0,0.0
57,20953,252731,2020-01-02,2020-01-05,0.00,0.00,0.0,2.0,0.0,1.0,0.0,0.0,1,0,0,1.000000,0.000000,0.0,1,NaN,NaN
58,20953,252731,2020-01-02,2020-01-11,0.00,0.00,0.0,1.0,0.0,1.0,0.0,0.0,1,0,0,1.000000,0.000000,0.0,1,NaN,NaN
59,20953,252732,2020-01-02,2020-01-02,2280.22,2280.22,3002.0,119.0,108335.0,117.0,0.0,0.0,112,2,0,0.982456,0.017544,0.0,114,0.0,0.0


In [244]:
df_source_no_amount = df_source[df_source['amount']==0]
df_source_amount = df_source[df_source['amount']>500]

In [231]:
df_source_no_amount['create_role_num'].value_counts()

1.0       23375
2.0        6259
3.0        2792
4.0        1691
5.0        1059
          ...  
367.0         1
454.0         1
370.0         1
98.0          1
1052.0        1
Name: create_role_num, Length: 372, dtype: int64

In [232]:
df_source_no_amount[df_source_no_amount['create_role_num'] ==1052]

,channel_id,source_id,create_time,reg_date,amount,paper_use_amount,click_num,down_num,pv,create_role_num,source_30_pay_sum,source_30_predict_pay,type_0_num,type_1_num,type_2_num,type_0_rate,type_1_rate,type_2_rate,source_role_num,30_ROI
254480,7162,261129,2020-01-21,2020-01-30,0.0,0.0,0.0,1084.0,0.0,1052.0,23082.0,29194.561164,934,118,23,0.868837,0.109767,0.021395,1075,inf


In [245]:
df_source_amount.head()

,channel_id,source_id,create_time,reg_date,amount,paper_use_amount,click_num,down_num,pv,create_role_num,source_30_pay_sum,source_30_predict_pay,type_0_num,type_1_num,type_2_num,type_0_rate,type_1_rate,type_2_rate,source_role_num,30_ROI,30_ROI_predict
0,20953,252731,2020-01-02,2020-01-02,1562.56,1562.56,1552.0,55.0,53605.0,51.0,0.0,0.000000,48,3,0,0.941176,0.058824,0.000000,51,0.000000,0.000000
59,20953,252732,2020-01-02,2020-01-02,2280.22,2280.22,3002.0,119.0,108335.0,117.0,0.0,0.000000,112,2,0,0.982456,0.017544,0.000000,114,0.000000,0.000000
173,20953,252732,2020-01-02,2020-01-03,700.59,700.59,1049.0,45.0,34087.0,45.0,0.0,0.000000,42,1,0,0.976744,0.023256,0.000000,43,0.000000,0.000000
223,20953,252733,2020-01-02,2020-01-02,2200.00,2200.00,666.0,219.0,53623.0,202.0,132.0,73.155306,189,3,1,0.979275,0.015544,0.005181,193,0.060000,0.033252
440,20953,252733,2020-01-02,2020-01-25,2200.00,2200.00,1394.0,450.0,103385.0,419.0,546.0,492.333356,361,20,1,0.945026,0.052356,0.002618,382,0.248182,0.223788


In [248]:
# 消耗大于500但30天回款为0的计划,消耗占比（消耗大于500的计划数）
df_source_amount[df_source_amount['30_ROI'] == 0]['amount'].sum()/df_source_amount['amount'].sum()

0.1526535339253292

In [246]:
# 消耗大于500但30天回款为0的计划数占比（消耗大于500的计划数）
df_source_amount[df_source_amount['30_ROI'] == 0].shape[0]/df_source_amount.shape[0]

0.3999621785173979

In [250]:
df_source_amount[df_source_amount['30_ROI']==0]['type_0_rate'].mean()

0.8758651632325264

In [240]:
df_source_amount[df_source_amount['30_ROI']>=0.7]['type_0_rate'].mean()

0.8439712528488261

In [241]:
df_source_amount[df_source_amount['30_ROI']<0.7]['type_0_rate'].mean()

0.861684137086141

In [252]:
df_1 = df_source_amount[['type_0_rate','type_1_rate','type_2_rate','30_ROI_predict','30_ROI']]

In [254]:
df_1['30_ROI'] = df_1['30_ROI'].apply(lambda x:0 if x==0 else 1)

In [256]:
df_1.head()

,type_0_rate,type_1_rate,type_2_rate,30_ROI_predict,30_ROI
0,0.941176,0.058824,0.000000,0.000000,0
59,0.982456,0.017544,0.000000,0.000000,0
173,0.976744,0.023256,0.000000,0.000000,0
223,0.979275,0.015544,0.005181,0.033252,1
440,0.945026,0.052356,0.002618,0.223788,1


In [278]:
df_1.corr()

,type_0_rate,type_1_rate,type_2_rate,30_ROI_predict,30_ROI
type_0_rate,1.000000,-0.970998,-0.241080,-0.031872,-0.073621
type_1_rate,-0.970998,1.000000,0.002054,-0.015006,0.009100
type_2_rate,-0.241080,0.002054,1.000000,0.194220,0.270985
30_ROI_predict,-0.031872,-0.015006,0.194220,1.000000,0.190295
30_ROI,-0.073621,0.009100,0.270985,0.190295,1.000000


In [257]:
df_1['30_ROI'].value_counts()

1    15865
0    10575
Name: 30_ROI, dtype: int64

In [292]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score,f1_score,accuracy_score,precision_score,classification_report
import lightgbm as lgb

In [262]:
df_train,df_test = train_test_split(df_1,test_size=0.2)

In [263]:
X_val, x_test, Y_val, y_test = train_test_split(df_train.drop('30_ROI',axis=1),df_train['30_ROI'],test_size=0.3)

In [264]:
# 不调参
params = {
"objective": "binary",
"boosting_type": "gbdt",
"learning_rate": 0.03,
"max_depth":3,
"num_leaves":35,
"min_child_samples":15,
"feature_fraction": 0.7,
"bagging_fraction":0.7,
"bagging_freq":10,
"lambda_l1":2,
"lambda_l2":2,
"metric": "auc",
# "is_unbalance":True
}

train_data = lgb.Dataset(X_val, label=Y_val)
val_data = lgb.Dataset(x_test, label=y_test, reference=train_data)
lgb_b = lgb.train(params, train_data, num_boost_round=8000, early_stopping_rounds=100, valid_sets=[train_data, val_data])

[1]	training's auc: 0.922357	valid_1's auc: 0.927281
Training until validation scores don't improve for 100 rounds
[2]	training's auc: 0.922402	valid_1's auc: 0.927326
[3]	training's auc: 0.922402	valid_1's auc: 0.927326
[4]	training's auc: 0.922402	valid_1's auc: 0.927326
[5]	training's auc: 0.922402	valid_1's auc: 0.927326
[6]	training's auc: 0.922459	valid_1's auc: 0.927478
[7]	training's auc: 0.922459	valid_1's auc: 0.927478
[8]	training's auc: 0.92252	valid_1's auc: 0.927478
[9]	training's auc: 0.92252	valid_1's auc: 0.927478
[10]	training's auc: 0.92252	valid_1's auc: 0.927478
[11]	training's auc: 0.924737	valid_1's auc: 0.929375
[12]	training's auc: 0.924745	valid_1's auc: 0.929399
[13]	training's auc: 0.924851	valid_1's auc: 0.929553
[14]	training's auc: 0.924852	valid_1's auc: 0.929553
[15]	training's auc: 0.924852	valid_1's auc: 0.929553
[16]	training's auc: 0.924861	valid_1's auc: 0.929534
[17]	training's auc: 0.924867	valid_1's auc: 0.929548
[18]	training's auc: 0.924901	va

[203]	training's auc: 0.929495	valid_1's auc: 0.930643
[204]	training's auc: 0.929479	valid_1's auc: 0.930894
[205]	training's auc: 0.92948	valid_1's auc: 0.930896
[206]	training's auc: 0.929485	valid_1's auc: 0.930899
[207]	training's auc: 0.929532	valid_1's auc: 0.930865
[208]	training's auc: 0.929528	valid_1's auc: 0.930852
[209]	training's auc: 0.929562	valid_1's auc: 0.930946
[210]	training's auc: 0.929658	valid_1's auc: 0.930992
[211]	training's auc: 0.92966	valid_1's auc: 0.93097
[212]	training's auc: 0.92966	valid_1's auc: 0.93097
[213]	training's auc: 0.929658	valid_1's auc: 0.930964
[214]	training's auc: 0.92976	valid_1's auc: 0.931026
[215]	training's auc: 0.929876	valid_1's auc: 0.931093
[216]	training's auc: 0.929877	valid_1's auc: 0.931094
[217]	training's auc: 0.930033	valid_1's auc: 0.931369
[218]	training's auc: 0.930034	valid_1's auc: 0.931353
[219]	training's auc: 0.930059	valid_1's auc: 0.93147
[220]	training's auc: 0.930069	valid_1's auc: 0.931449
[221]	training's 

In [265]:
target_test = df_test['30_ROI']
features_test = df_test.drop('30_ROI', axis=1)
y_predict = lgb_b.predict(features_test)

In [309]:
sessoin = pd.DataFrame(y_predict, columns=['pay_30'])
sessoin['pay_30']=sessoin['pay_30'].apply(lambda x:0 if x<0.3 else 1)
y_predict_val = sessoin['pay_30'].values
print(recall_score(target_test, y_predict_val,average=None))
print(precision_score(target_test, y_predict_val))
print(accuracy_score(target_test, y_predict_val))
print(f1_score(target_test, y_predict_val))

[0.60842803 0.91026448]
0.777568585260893
0.789712556732224
0.8387003191180737


In [310]:
target_names = ['0', '1']
print(classification_report(target_test, y_predict_val, target_names=target_names))

              precision    recall  f1-score   support

           0       0.82      0.61      0.70      2112
           1       0.78      0.91      0.84      3176

    accuracy                           0.79      5288
   macro avg       0.80      0.76      0.77      5288
weighted avg       0.79      0.79      0.78      5288



In [311]:
joblib.dump(lgb_b,'./source_lgb_b.pkl')

['./source_lgb_b.pkl']